In [125]:
import torch
from torch import nn, optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from typing import Optional, Tuple
from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset # 텐서데이터셋
from torch.utils.data import DataLoader # 데이터로더

In [ ]:
device = torch.device('cuda')
data_csv = "machine_649" + ".csv"
use_RevIN = False

data = pd.read_csv(f"./cpu_mem/{data_csv}")
#data['time'] = pd.to_datetime(data['time'])
#data.set_index('time', inplace=True)
data.set_index('date', inplace=True)
print(data.dtypes)
print(data)

In [127]:
history_size = 48
predict_size = 96
batch = 4
hidden_size = 256
n_layers = 4
n_epoch = 500
lr = 0.0003
train_split = predict_size * batch

In [128]:
class RevIN(nn.Module):
    def __init__(self, num_features: int, eps=1e-5, affine=True):
        """
        :param num_features: the number of features or channels
        :param eps: a value added for numerical stability
        :param affine: if True, RevIN has learnable affine parameters
        """
        super(RevIN, self).__init__()
        self.num_features = num_features
        self.eps = eps
        self.affine = affine
        if self.affine:
            self._init_params()

    def forward(self, x, mode:str):
        if mode == 'norm':
            self._get_statistics(x)
            x = self._normalize(x)
        elif mode == 'denorm':
            x = self._denormalize(x)
        else: raise NotImplementedError
        return x

    def _init_params(self):
        # initialize RevIN params: (C,)
        self.affine_weight = nn.Parameter(torch.ones(self.num_features))
        self.affine_bias = nn.Parameter(torch.zeros(self.num_features))

    def _get_statistics(self, x):
        dim2reduce = tuple(range(1, x.ndim-1))
        self.mean = torch.mean(x, dim=dim2reduce, keepdim=True).detach()
        self.stdev = torch.sqrt(torch.var(x, dim=dim2reduce, keepdim=True, unbiased=False) + self.eps).detach()

    def _normalize(self, x):
        x = x - self.mean
        x = x / self.stdev
        if self.affine:
            x = x * self.affine_weight
            x = x + self.affine_bias
        return x

    def _denormalize(self, x):
        if self.affine:
            x = x - self.affine_bias
            x = x / (self.affine_weight + self.eps*self.eps)
        x = x * self.stdev
        x = x + self.mean
        return x

In [129]:
class LSTMCell(nn.Module):
    def __init__(self, input_size : int, hidden_size : int):
        super(LSTMCell, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.hidden_lin = nn.Linear(hidden_size, 4 * hidden_size)
        self.input_lin = nn.Linear(input_size, 4 * hidden_size, bias=False)

    def forward(self, x, h_in, c_in):
        X = self.input_lin(x) + self.hidden_lin(h_in) # 입력과 은닉 상태를 선형 변환 후 더함
        i, f, g, o = X.chunk(4, dim=-1)

        i = torch.sigmoid(i)
        f = torch.sigmoid(f)
        g = torch.tanh(g)
        o = torch.sigmoid(o)

        c_next = c_in * f + i * g
        h_next = o * torch.tanh(c_next)

        return h_next, c_next
    
class MY_LSTM(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, n_layers: int, use_RevIN: bool):
        super(MY_LSTM, self).__init__()
        self.n_layers = n_layers
        self.hidden_size = hidden_size
        self.use_Revin = use_RevIN
        self.cells = nn.ModuleList(
            [LSTMCell(input_size=input_size, hidden_size=hidden_size)] +
            [LSTMCell(input_size=hidden_size, hidden_size=hidden_size) for _ in range(n_layers - 1)]
        )
        self.linear = nn.Linear(self.hidden_size, 1)
        self.revin = RevIN(1)

    def forward(self, x: torch.Tensor, state: Optional[Tuple[torch.Tensor, torch.Tensor]] = None):
        batch_size, seq_len, _ = x.shape
        if self.use_Revin:
            x = self.revin(x, "norm")

        if state is None:
            h = [x.new_zeros(batch_size, self.hidden_size) for _ in range(self.n_layers)]
            c = [x.new_zeros(batch_size, self.hidden_size) for _ in range(self.n_layers)]
        else:
            h, c = state
            h, c = list(torch.unbind(h)), list(torch.unbind(c))

        outputs = []  # 각 time step의 출력을 담는 리스트
        for t in range(batch_size):
            inp = x[t, :].squeeze(-1)  # 각 시점의 입력
            for layer in range(self.n_layers):
                h[layer], c[layer] = self.cells[layer](inp, h[layer], c[layer])
                inp = h[layer]
            outputs.append(self.linear(h[-1]))  # 각 time step에서 마지막 layer의 hidden state를 사용해 예측
        
        outputs = torch.stack(outputs, dim=0).squeeze(1)  # 모든 time step의 예측을 쌓음

        if self.use_Revin:
            outputs = self.revin(outputs, "denorm").squeeze(1)

        h = torch.stack(h)
        c = torch.stack(c)

        return outputs, (h, c)


In [130]:
def univariate_data(dataset, start_index, end_index, history_size, target_size, single_step=False):
    datas = []
    labels = []

    start_index = start_index + history_size

    if end_index == None:
        end_index = len(dataset) - target_size
    
    for i in range(start_index, end_index):
        indices = range(i - history_size, i)
        datas.append(np.reshape(dataset[indices], (history_size,1)))

        if single_step: # 단기 예측
            labels.append(dataset[i]) # start_index + history_size + target_size
        else: # 장기 예측
            labels.append(dataset[i:i + target_size]) # i ~ i + target_size - 1

    return np.array(datas), np.array(labels)

def multivariate_data(dataset, start_index, end_index, history_size, target_size, step, single_step=False):
    datas = []
    labels = []

    start_index = start_index + history_size

    if end_index == None:
        end_index = len(dataset) - target_size
    
    for i in range(start_index, end_index):
        indices = range(i - history_size, i, step)
        datas.append(dataset[indices])

        if single_step: # 단기 예측
            labels.append(dataset[i + target_size])
        else: # 장기 예측
            labels.append(dataset[i:i + target_size])
    return np.array(datas), np.array(labels)

def create_time_steps(length):
    if length >= 0:
        return range(0, length)
    else:
        return range(length + 1, 1)

def show_plot(plot_data, delta, title):
    labels = ["history", "true future", "baseline", "mean"]
    marker = ["-", "r-", "g-", "yx"]
    time_steps = create_time_steps(-plot_data[0].shape[0])

    plt.title(title)
    for i, x in enumerate(plot_data):
        if i == 3:
            plt.plot(1, plot_data[i], marker[i], label=labels[i])
        elif i:
            plt.plot(create_time_steps(x.shape[0]), plot_data[i], marker[i], label=labels[i])
        else:
            plt.plot(time_steps, plot_data[i].flatten(), marker[i], label=labels[i])

    plt.legend()
    plt.axis('auto')
    plt.xlabel('time-steps')
    plt.show()
    return plt  

In [ ]:
x_train_uni, y_train_uni = univariate_data(data['OT'], 0, train_split, history_size, predict_size, True)
x_test_uni, y_test_uni = univariate_data(data['OT'], train_split, None, history_size, predict_size, True)

x_train_uni = torch.FloatTensor(x_train_uni).to(device)
y_train_uni = torch.FloatTensor(y_train_uni).to(device)
x_test_uni = torch.FloatTensor(x_test_uni).to(device)
y_test_uni = torch.FloatTensor(y_test_uni).to(device)

dataset = TensorDataset(x_train_uni, y_train_uni)
dataloader = DataLoader(dataset, batch_size=predict_size)

test_dataset = TensorDataset(x_test_uni, y_test_uni)
test_dataloader = DataLoader(test_dataset, batch_size=predict_size)

train, test = next(iter(test_dataloader))
print(train.shape, test.shape)

# data_len = histroy size
show_plot([train[1].squeeze(-1).cpu(), test.cpu(), test.cpu(), test[0].cpu()], 0, "title")

In [132]:
class RMSELoss(nn.Module):
    def __init__(self):
        super(RMSELoss,self).__init__()
        self.mse = nn.MSELoss()
        self.eps = 1e-7

    def forward(self,y,y_hat):
        return torch.sqrt(self.mse(y,y_hat) + self.eps)

In [133]:
class LSTM(nn.Module):
    def __init__(self, input_size, hidden_size, num_classes, n_layers):
        super(LSTM, self).__init__()
        self.hidden_size = hidden_size
        self.num_layers = n_layers
        self.lstm = nn.LSTM(input_size, hidden_size, n_layers, batch_first=True)
        self.output_layer = nn.Linear(hidden_size, num_classes) # 입력 : hidden_size, 출력 : num_classes

    def forward(self, x):
        hidden_states = torch.zeros(self.num_layers, x.shape[0], self.hidden_size) # hidden_layer 개수, sample 개수, hidden_layer 크기
        cell_states = torch.zeros(self.num_layers, x.shape[0], self.hidden_size)
        out, (h, c) = self.lstm(x, (hidden_states, cell_states)) # input : x, 초기 상태 : (hidden_states, cell_states), out : 모든 time step(sequence)의 출력, _ : 은닉 상태 및 셀 상태
        out = self.output_layer(out) # 각 입력 시퀀스에 대해 마지막 time step의 출력
        return out, (h, c)

In [134]:
model1 = LSTM(input_size=history_size, hidden_size=hidden_size, n_layers=n_layers, num_classes=1).to(device)
model2 = MY_LSTM(input_size=history_size, hidden_size=hidden_size, n_layers=n_layers, use_RevIN=use_RevIN).to(device)
optimizer = optim.Adam(model2.parameters(), lr)

In [135]:
import time

def train_model(model, train_df, num_epochs = 5, lr = 0.001, verbose = 2, patience = 50):
    model.train()
     
    criterion = nn.MSELoss()
    optimizer = optim.Adam(model.parameters(), lr)
    
    # epoch마다 loss 저장
    train_hist = np.zeros(num_epochs)

    for epoch in range(num_epochs):
        avg_cost = 0
        total_batch = len(train_df)
        
        start = time.time()
        for batch_idx, samples in enumerate(train_df):
            # seq별 hidden state reset
            h, c = None, None
            
            x_train, y_train = samples
            x_train.squeeze(0)

            # H(x) 계산
            # outputs, _ = model(x_train)
            if h is not None and c is not None:
                h = h.detach()
                c = c.detach()
                
            # 모델 예측
            outputs, _ = model(x_train, (h, c)) if h is not None and c is not None else model(x_train)
                
            # cost 계산
            loss = criterion(outputs[:, -1, :], y_train)   
            
            # cost로 H(x) 개선
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            
            avg_cost += loss/total_batch
               
        train_hist[epoch] = avg_cost        
        
        if epoch % verbose == 0:
            print('Epoch:', '%04d' % (epoch), 'train loss :', '{:.4f}'.format(avg_cost), f'time: {time.time() - start}')
            show_plot([x_train[1].squeeze(-1).cpu(), y_train.cpu(), outputs[:, -1, :].detach().cpu().numpy()], 0, "title")
            
        # patience번째 마다 early stopping 여부 확인
        if (epoch % patience == 0) & (epoch != 0):
            
            # loss가 커졌다면 early stop
            if train_hist[epoch-patience] < train_hist[epoch]:
                print('\n Early Stopping')
                
                break
            
    return model.eval(), train_hist

In [136]:
# train(x_train_uni., y_train_uni., model, criterion, optimizer, n_epoch)
model, train_hist = train_model(model2, dataloader, num_epochs = n_epoch, lr = lr, verbose = 1, patience = 10)

In [137]:
# epoch별 손실값
fig = plt.figure(figsize=(10, 4))
plt.plot(train_hist, label="Training loss")
plt.legend()
plt.savefig(f"./Metrics/loss/Timeseries_LSTM_{data_csv}_{use_RevIN}_{history_size}_{predict_size}_RevIN_Training_Loss.png")
plt.show()

In [138]:
# 모델 저장    
import os
PATH = f"./checkpoint/Timeseries_LSTM_{data_csv}_{use_RevIN}_{history_size}_{predict_size}_RevIN"
if os.path.exists(PATH + ".pth") == False:
    torch.save(model.state_dict(), PATH + ".pth")

# 불러오기
# model1 = LSTM(input_size=1, hidden_size=hidden_size, n_layers=n_layers, num_classes=1)
model = MY_LSTM(input_size=history_size, hidden_size=hidden_size, n_layers=n_layers, use_RevIN=use_RevIN).to(device)
model.load_state_dict(torch.load(PATH + ".pth"), strict=False)
model.eval()

In [139]:
def RSE(pred, true):
    return np.sqrt(np.sum((true-pred)**2)) / np.sqrt(np.sum((true-true.mean())**2))

def CORR(pred, true):
    u = ((true-true.mean(0))*(pred-pred.mean(0))).sum(0) 
    d = np.sqrt(((true-true.mean(0))**2*(pred-pred.mean(0))**2).sum(0))
    return (u/d).mean(-1)

def MAE(pred, true):
    return np.mean(np.abs(pred-true))

def MSE(pred, true):
    return np.mean((pred-true)**2)

def RMSE(pred, true):
    return np.sqrt(MSE(pred, true))

def MAPE(pred, true):
    return np.mean(np.abs((pred - true) / true))

def MSPE(pred, true):
    return np.mean(np.square((pred - true) / true))

def metric(pred, true):
    mae = MAE(pred, true)
    mse = MSE(pred, true)
    rse = RSE(pred, true)
    rmse = RMSE(pred, true)
    mape = MAPE(pred, true)
    mspe = MSPE(pred, true)
    
    f = open(f"./Metrics/Timeseries_LSTM_{data_csv}_{use_RevIN}_{history_size}_{predict_size}_RevIN.txt", 'w')
    f.write('MAE SCORE : ' + str(mae) + "\n")
    f.write('MSE SCORE : ' + str(mse) + "\n")
    f.write('RSE SCORE : ' + str(rse) + "\n")
    f.write('RMSE SCORE : ' + str(rmse) + "\n")
    f.write('MAPE SCORE : ' + str(mape) + "\n")
    f.write('MSPE SCORE : ' + str(mspe) + "\n")
    f.close()

# 예측 테스트
with torch.no_grad(): 
    pred = []
    true = []
    for pr in test_dataloader:
        test_x, test_y = pr
        if test_x.shape[0] < predict_size:
            continue
        predicted, _ = model(test_x.squeeze(0))
        pred.append(predicted[:, -1, :].squeeze(-1).cpu())
        true.append(test_y.cpu())
    true = np.array(true)
    pred = np.array(pred)
    metric(pred, true)